<h2 style="text-align: center"><i>The Interaction Between Two Atoms</i></h2>

**Author:** 2023WS_61714

**Date:** November 22, 2023

---

**Goal:** 
The Goal is to apply scientific computing knowledge by extending previous work on the Lennard-Jones equation [[1]](#1), implementing <br /> an alternative equation for nonbonded interactions, and exploring the Pandas library [[3]](#3) for data manipulation and analysis.

<div style="text-align: center">

<a id="lje">
<h4>Lennard-Jones Equation:</h4>
</a>

</div>

<div style="text-align: center">

#### $V_{LJ}(r) = 4\varepsilon \ \left[\left(\frac {\sigma} {r}\right)^{12} -\left(\frac {\sigma} {r}\right)^{6} \right]$

</div>

<div style="text-align: center">

<a id="eee">
<h4>Yang et al.'s Expanded Exponential Equation: </h4>
</a>

</div>

<div style="text-align: center">

#### $$V_{Exp}(r)=\varepsilon \left[  e^{\alpha (1-\frac{r}{\sigma})} - \left( \left(\frac{r}{\sigma} \right)^4 - 2 \left(\frac{r}{\sigma} \right)^2 + 3\right) e^{(\frac{\alpha}{2})(1-\frac{r}{\sigma})}  \right]$$

</div>

---

**References**

<a id="1">[1]</a> Lennard-Jones Equation: https://en.wikipedia.org/wiki/Lennard-Jones_potential.
<br />
<a id="2">[2]</a> Hartree to kj/mol: http://wild.life.nctu.edu.tw/class/common/energy-unit-conv-table.html
<br />
<a id="3">[3]</a> Python Pandas: https://johnfoster.pge.utexas.edu/numerical-methods-book/ScientificPython_Pandas.html
<br />
<a id="4">[4]</a> Python Pandas DataFrame drop functions: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
<br />
<a id="5">[5]</a> Python Pandas DataFrame apply function: https://www.geeksforgeeks.org/apply-function-to-every-row-in-a-pandas-dataframe/
<br />
<a id="6">[6]</a> extended exponential Equation from Yang et al: [van der Waals Function for Molecular Mechanics](../docs/YangSD2020.pdf)

**Import**

Let's import the essential libraries needed for the assignment: Pandas and Python's built-in math library. <br />So that we can use them later in the code blocks.

In [64]:
import pandas as pd
import numpy as np
import math

---

<h3> Task 1: </h3>

We need utilize `Pandas` to load the $QM$ target data for $Ar_2$ from the file ([CybulskiT1999_Ar2.csv](CybulskiT1999_Ar2.csv)), making sure to **eliminate duplicates** and **missing rows**. <br /> 
After that, we need to **convert the numbers** in the potential energy column **from Hartree to kJ/mol** [[2]](#2).

In [65]:
data_frame = pd.read_csv('../docs/CybulskiT1999_Ar2.csv', sep=";")
data_frame

,R (Å),V(r) (Hartree)
0,3.000,0.003055
1,3.250,0.000518
2,3.500,-0.000279
3,3.500,-0.000279
4,3.600,NaN
5,3.750,-0.000441
6,3.775,-0.000442
7,3.800,-0.000441
8,3.850,-0.000436
9,3.950,NaN


We can use the Pandas [[4]](#4) `.drop_duplicates()` function to **eliminate duplicate entries** in our data frame. <br /> Additionally, the `.dropna()` function allows us to **remove missing or NaN values**

In [66]:
data_frame = data_frame.drop_duplicates()
data_frame = data_frame.dropna()
data_frame

,R (Å),V(r) (Hartree)
0,3.000,0.003055
1,3.250,0.000518
2,3.500,-0.000279
5,3.750,-0.000441
6,3.775,-0.000442
7,3.800,-0.000441
8,3.850,-0.000436
10,4.000,-0.000400
11,4.250,-0.000313
12,4.500,-0.000232


Now, we convert the values **from Hartree to kJ/mol** [[2]](#2) by multiplying the values with `2625.5`.

In [67]:
data_frame['V(r) (Hartree)'] *= 2625.5
data_frame.rename({'V(r) (Hartree)' : 'V(r) (kJ/mol)'}, axis='columns', inplace=True)

data_frame

,R (Å),V(r) (kJ/mol)
0,3.000,8.020850
1,3.250,1.360298
2,3.500,-0.732462
5,3.750,-1.157373
6,3.775,-1.160130
7,3.800,-1.158686
8,3.850,-1.145269
10,4.000,-1.049150
11,4.250,-0.822097
12,4.500,-0.609904


---

<h3> Task 2: </h3>

In the task we have to calculate the **Lennard-Jones potential energy** [Eq. 1](#lje) and the <br /> **exponential potential energy** [Eq. 2](#eee) using the distances given in the $QM$ target data.

<center> These specific Argon atom parameters have been provided in the assignment paper: </center>

<center>

| $Species$            | $\epsilon (kJ/mol)$ | $\sigma (Å)$ | $α$   |
| ------------------ | -------------------- | ------------ | ----- |
| $Ar_2$             | 1.178                | 3.75         | 13.18 |

</center>

We will assign these values in a separate code block so that we can use them later

In [68]:
epsilon = 1.178 # kJ/mol
sigma = 3.75 #Å
alpha = 13.18

To calculate the non-bonded potential energy between two atoms, <br /> we use the Lennard-Jones equation [[1]](#1), which is represented by the following Python function:

In [69]:
def lennard_jones_eq(r:float, epsilon:float, sigma:float):
    return (4 * epsilon * (((sigma / r) ** 12) - ((sigma / r) ** 6)))

Next, we apply the `lennard_jones_eq()` function to the **R (Å)** column in the dataframe, <br/> using pandas' built-in function `.apply()` [[5]](#5). Additionally, we add a new column titled: **Lennard-Jones Potential Energy (kJ/mol)** to showcase the values.

In [70]:
data_frame['Lennard-Jones potential energy (kJ/mol)'] = data_frame.apply(lambda row: lennard_jones_eq(r=row['R (Å)'], 
epsilon=epsilon, sigma=sigma), axis=1)
data_frame

,R (Å),V(r) (kJ/mol),Lennard-Jones potential energy (kJ/mol)
0,3.000,8.020850,50.593771
1,3.250,1.360298,15.121296
2,3.500,-0.732462,3.655327
5,3.750,-1.157373,0.000000
6,3.775,-1.160130,-0.176962
7,3.800,-1.158686,-0.332475
8,3.850,-1.145269,-0.587738
10,4.000,-1.049150,-1.027134
11,4.250,-0.822097,-1.174281
12,4.500,-0.609904,-1.049557


To compute potential energy using the **extended exponential Equation from Yang et al** [[5]](#5), which <br /> is represented by the following Python function:

In [71]:
def expanded_exponential_eq(r, epsilon, sigma, alpha):
    term1 = epsilon * (np.exp(alpha * (1 - r / sigma)))
    term2 = ((r / sigma) ** 4 - 2 * (r / sigma) ** 2 + 3) * np.exp(0.5 * alpha * (1 - r / sigma))
    return term1 - term2

Finally, we apply the `expanded_exponential_eq()` function to the **R (Å)** column in the dataframe, `.apply()` [[5]](#5) again <br /> and  we add a new column titled: **extended exponential Equation (kJ/mol)** to showcase the values.

In [72]:
data_frame['Exponential potential energy (kJ/mol)'] = data_frame.apply(lambda row: expanded_exponential_eq(r=row['R (Å)'],
epsilon=epsilon, sigma=sigma, alpha=alpha), axis=1)
data_frame

,R (Å),V(r) (kJ/mol),Lennard-Jones potential energy (kJ/mol),Exponential potential energy (kJ/mol)
0,3.000,8.020850,50.593771,8.485593
1,3.250,1.360298,15.121296,1.864296
2,3.500,-0.732462,3.655327,-0.292866
5,3.750,-1.157373,0.000000,-0.822000
6,3.775,-1.160130,-0.176962,-0.835297
7,3.800,-1.158686,-0.332475,-0.844271
8,3.850,-1.145269,-0.587738,-0.851227
10,4.000,-1.049150,-1.027134,-0.811899
11,4.250,-0.822097,-1.174281,-0.661067
12,4.500,-0.609904,-1.049557,-0.502761
